In [ ]:
json_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'

In [1]:
import requests
from requests import Response
def get_response() -> Response | None:
    json_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'
    try:
        response:Response =requests.get(json_url)
        response.raise_for_status()
        if response.status_code == 200:
            print("下載成功")
            return response
        else:
            print("下載失敗")
            return None
    except Exception as e:
        print(e)
        print("連線失敗")
        return None

In [ ]:
response:Response | None = get_response()
if response:
    rawString:str = response.text
    print(rawString)

In [3]:
from pydantic import BaseModel,Field,field_validator

class Site(BaseModel):
    站點名稱:str = Field(alias='sitename')
    縣市:str = Field(alias='county')
    aqi:float
    時間:str = Field(alias='datacreationdate')
    pm25:float = Field(alias='pm2.5')

    @field_validator("pm25",mode='before')
    @classmethod
    def empty_to_zero(cls,value):
        if value=='':
            return '0.0'
        else:
            return value
        
class Sites(BaseModel):
    records:list[Site]

In [6]:
data:Sites = Sites.model_validate_json(rawString)

for item in data.records:
    print(item)

站點名稱='屏東(枋山)' 縣市='屏東縣' aqi=34.0 時間='2024-03-28 20:00' pm25=7.0
站點名稱='新北(樹林)' 縣市='新北市' aqi=67.0 時間='2024-03-28 20:00' pm25=18.0
站點名稱='屏東(琉球)' 縣市='屏東縣' aqi=40.0 時間='2024-03-28 20:00' pm25=10.0
站點名稱='臺南(麻豆)' 縣市='臺南市' aqi=59.0 時間='2024-03-28 20:00' pm25=17.0
站點名稱='高雄(湖內)' 縣市='高雄市' aqi=44.0 時間='2024-03-28 20:00' pm25=7.0
站點名稱='彰化(員林)' 縣市='彰化縣' aqi=71.0 時間='2024-03-28 20:00' pm25=22.0
站點名稱='大城' 縣市='彰化縣' aqi=61.0 時間='2024-03-28 20:00' pm25=13.0
站點名稱='富貴角' 縣市='新北市' aqi=58.0 時間='2024-03-28 20:00' pm25=12.0
站點名稱='麥寮' 縣市='雲林縣' aqi=64.0 時間='2024-03-28 20:00' pm25=10.0
站點名稱='關山' 縣市='臺東縣' aqi=28.0 時間='2024-03-28 20:00' pm25=11.0
站點名稱='馬公' 縣市='澎湖縣' aqi=44.0 時間='2024-03-28 20:00' pm25=16.0
站點名稱='金門' 縣市='金門縣' aqi=97.0 時間='2024-03-28 20:00' pm25=30.0
站點名稱='馬祖' 縣市='連江縣' aqi=126.0 時間='2024-03-28 20:00' pm25=45.0
站點名稱='埔里' 縣市='南投縣' aqi=65.0 時間='2024-03-28 20:00' pm25=26.0
站點名稱='復興' 縣市='高雄市' aqi=59.0 時間='2024-03-28 20:00' pm25=16.0
站點名稱='永和' 縣市='新北市' aqi=51.0 時間='2024-03-28 20:00' pm25=12.0
站點名稱='竹山' 縣市='南投